In [9]:
df = spark.sql("SELECT * FROM bing_lake_db.tbl_latest_news")
display(df)

StatementMeta(, 8e17928f-2b1b-4ae3-9bcf-50313a093355, 11, Finished, Available)

SynapseWidget(Synapse.DataFrame, 0ef70eeb-9820-4967-82d5-c8a15143ec10)

In [10]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

StatementMeta(, 8e17928f-2b1b-4ae3-9bcf-50313a093355, 12, Finished, Available)

In [11]:
model = (AnalyzeText()
        .setTextCol("description")
        .setKind("SentimentAnalysis")
        .setOutputCol("response")
        .setErrorCol("error"))

StatementMeta(, 8e17928f-2b1b-4ae3-9bcf-50313a093355, 13, Finished, Available)

In [12]:
result = model.transform(df)

StatementMeta(, 8e17928f-2b1b-4ae3-9bcf-50313a093355, 14, Finished, Available)

In [13]:
display(result)

StatementMeta(, 8e17928f-2b1b-4ae3-9bcf-50313a093355, 15, Finished, Available)

SynapseWidget(Synapse.DataFrame, 88e7e935-7dc2-454b-ac43-9becb9155cd7)

In [14]:
from pyspark.sql.functions import col

sentiment_df = result.withColumn("sentiment", col("response.documents.sentiment"))

StatementMeta(, 8e17928f-2b1b-4ae3-9bcf-50313a093355, 16, Finished, Available)

In [15]:
sentiment_df_final = sentiment_df.drop("error","response")

StatementMeta(, 8e17928f-2b1b-4ae3-9bcf-50313a093355, 17, Finished, Available)

In [16]:
display(sentiment_df_final)

StatementMeta(, 8e17928f-2b1b-4ae3-9bcf-50313a093355, 18, Finished, Available)

SynapseWidget(Synapse.DataFrame, 3d76bdf2-a52c-4e5b-a34e-2123426b898f)

In [17]:
from pyspark.sql.utils import AnalysisException

try:

    table_name = 'bing_lake_db.tbl_sentiment_analysis'

    sentiment_df_final.write.format("delta").saveAsTable(table_name)

except AnalysisException:

    print("Table Already Exists")

    sentiment_df_final.createOrReplaceTempView("vw_sentiment_df_final")

    spark.sql(f"""  MERGE INTO {table_name} target_table 
                    USING vw_sentiment_df_final source_view

                    ON source_view.url = target_table.url

                    WHEN MATCHED AND
                    source_view.title <> target_table.title OR
                    source_view.description <> target_table.description OR
                    source_view.category <> target_table.category OR
                    source_view.image <> target_table.image OR
                    source_view.provider <> target_table.provider OR
                    source_view.datePublished <> target_table.datePublished

                    THEN UPDATE SET *
                    WHEN NOT MATCHED THEN INSERT *
                """)

StatementMeta(, 8e17928f-2b1b-4ae3-9bcf-50313a093355, 19, Finished, Available)